In [16]:
from pathlib import Path
from functools import reduce
import pandas as pd
import os

In [6]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

2023-07-17 05:25:22.266865: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 1


2023-07-17 05:25:23.774276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:23.791298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:23.791507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.


In [7]:
!nvidia-smi

Mon Jul 17 05:25:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.06              Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  | 00000000:05:00.0  On |                  N/A |
|  0%   44C    P8              15W / 200W |   7845MiB /  8192MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
target = 'header'
version = 3
step2_train_path = Path('/tf/datasets/step2_new/'+target+'/train')
step2_validation_path = Path('/tf/datasets/step2_new/'+target+'/validation')

In [9]:
print(step2_train_path, step2_validation_path)
!pwd
!pip list

/tf/datasets/step2_new/header/train /tf/datasets/step2_new/header/validation
/tf/notebook/step2
Package                      Version
---------------------------- --------------------
absl-py                      1.4.0
anyio                        3.7.1
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
array-record                 0.4.0
arrow                        1.2.3
asttokens                    2.2.1
astunparse                   1.6.3
attrs                        23.1.0
backcall                     0.2.0
beautifulsoup4               4.12.2
bleach                       6.0.0
cachetools                   5.3.1
certifi                      2022.12.7
cffi                         1.15.1
chardet                      4.0.0
charset-normalizer           3.2.0
click                        8.1.4
cmake                        3.26.4
comm                         0.1.3
contourpy                    1.1.0
cycler                       0.10.0
dbus-python                  1.2.16


In [10]:
labels = [path.name for path in step2_train_path.iterdir()]
def prepare_data_frame(step2_path):
    files = {label.name: [file for file in label.iterdir()] for label in step2_path.iterdir()}
    
    file_paths = reduce(lambda x,y: x+y, files.values())
    file_names = list(map(lambda x: x.name, file_paths))
    name_to_path_dict = dict(zip(file_names, map(lambda x: str(x), file_paths)))
    file_path_series = pd.Series(name_to_path_dict, dtype='string', name='file_path')
    
    label_series_list = [
        pd.Series(1.0, index=map(lambda x: x.name, files[label]), name=label)
        for label in labels
    ]
    df = reduce(lambda x, y: pd.merge(x, y, how='outer', left_index=True, right_index=True), label_series_list)
    df = df.fillna(value=0.0)
    
    df_with_file_path = pd.merge(df, file_path_series, left_index=True, right_index=True)
    
    return df_with_file_path

In [11]:
train_df = prepare_data_frame(step2_train_path)
validation_df = prepare_data_frame(step2_validation_path)
print(train_df, validation_df)

                                                    logo_center  logo_left  \
cys_20220928_181059_01.jpg                                  1.0        0.0   
cys_20220929_101001_01.jpg                                  1.0        0.0   
cys_20220929_110353_01.jpg                                  0.0        1.0   
cys_20220929_111453_02.jpg                                  1.0        0.0   
cys_20220929_112345_02.jpg                                  1.0        0.0   
...                                                         ...        ...   
hyw_screencapture-wpbingosite-wordpress-ruper-h...          0.0        1.0   
hyw_screencapture-wpbingosite-wordpress-ruper-h...          0.0        1.0   
hyw_screencapture-wpbingosite-wordpress-vatage-...          0.0        1.0   
hyw_screencapture-yonifit-co-kr-2022-09-30-09_1...          1.0        0.0   
hyw_screencapture-zierli-2022-09-28-16_29_23_01...          1.0        0.0   

                                                               

In [12]:
import tensorflow as tf
import tensorflow_hub as hub

In [13]:
train_ds = tf.data.Dataset.from_tensor_slices({**train_df})
print( train_ds )
validation_ds = tf.data.Dataset.from_tensor_slices({**validation_df})
print( validation_ds )

<_TensorSliceDataset element_spec={'logo_center': TensorSpec(shape=(), dtype=tf.float64, name=None), 'logo_left': TensorSpec(shape=(), dtype=tf.float64, name=None), 'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}>
<_TensorSliceDataset element_spec={'logo_center': TensorSpec(shape=(), dtype=tf.float64, name=None), 'logo_left': TensorSpec(shape=(), dtype=tf.float64, name=None), 'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}>


2023-07-17 05:25:35.762388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:35.762672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:35.762850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:36.474134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:05:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-17 05:25:36.474352: I tensorflow/compile

In [14]:
def load_image(sample):
    file_path = sample['file_path']
    image_bytes = tf.io.read_file(file_path)
    image = tf.io.decode_image(image_bytes, channels=3, expand_animations=False)
    sample['image'] = image
    return sample

def preprocess_image(sample):
    image = tf.image.convert_image_dtype(sample['image'], tf.float32)
    image = tf.image.resize(image, [224, 224])
    sample['image'] = image
    return sample

def transform_data_for_keras_fit(sample):
    return sample['image'], tuple(sample[label] for label in labels)

In [17]:
trainable_ds = train_ds.map(load_image).map(preprocess_image).map(transform_data_for_keras_fit).batch(32)
testable_ds = validation_ds.map(load_image).map(preprocess_image).map(transform_data_for_keras_fit).batch(32)
print( trainable_ds )
print( testable_ds )

TypeError: in user code:

    File "/tmp/ipykernel_121/1732407592.py", line 3, in load_image  *
        _, file_ext = os.path.splitext(file_path)
    File "/usr/lib/python3.8/posixpath.py", line 118, in splitext  **
        p = os.fspath(p)

    TypeError: expected str, bytes or os.PathLike object, not Tensor


In [12]:
inputs = tf.keras.Input(shape=(224, 224, 3), name="image")
efficientnet_v2 = hub.KerasLayer('https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2', 
                                 trainable=False)
net = efficientnet_v2(inputs)
# outputs = [tf.keras.layers.Dense(1, name='dense_'+label)(net) for label in labels]
outputs = [tf.keras.layers.Dense(1, name=label, activation='sigmoid')(net) for label in labels]

efficientnet_v2_step2 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [13]:
efficientnet_v2_step2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image (InputLayer)          [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 keras_layer (KerasLayer)    (None, 1536)                 1293062   ['image[0][0]']               
                                                          2                                       
                                                                                                  
 2logo_center (Dense)        (None, 1)                    1537      ['keras_layer[0][0]']         
                                                                                                  
 2logo_left (Dense)          (None, 1)                    1537      ['keras_layer[0][0]']     

In [14]:
efficientnet_v2_step2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=[tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy_'+label) for label in labels],
    metrics=tf.keras.metrics.BinaryAccuracy(),
)

In [17]:
history = efficientnet_v2_step2.fit(
    trainable_ds, epochs=40, validation_data=testable_ds,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/fit')]
)

Epoch 1/40
946/946 [==============================] - 184s 194ms/step - loss: 0.5019 - 2logo_center_loss: 0.2516 - 2logo_left_loss: 0.2503 - 2logo_center_binary_accuracy: 0.8955 - 2logo_left_binary_accuracy: 0.8959 - val_loss: 1.1875 - val_2logo_center_loss: 0.5944 - val_2logo_left_loss: 0.5931 - val_2logo_center_binary_accuracy: 0.8019 - val_2logo_left_binary_accuracy: 0.8038
Epoch 2/40
946/946 [==============================] - 226s 238ms/step - loss: 0.5015 - 2logo_center_loss: 0.2514 - 2logo_left_loss: 0.2501 - 2logo_center_binary_accuracy: 0.8955 - 2logo_left_binary_accuracy: 0.8960 - val_loss: 1.1896 - val_2logo_center_loss: 0.5954 - val_2logo_left_loss: 0.5942 - val_2logo_center_binary_accuracy: 0.8020 - val_2logo_left_binary_accuracy: 0.8037
Epoch 3/40
946/946 [==============================] - 73s 77ms/step - loss: 0.5011 - 2logo_center_loss: 0.2512 - 2logo_left_loss: 0.2499 - 2logo_center_binary_accuracy: 0.8957 - 2logo_left_binary_accuracy: 0.8961 - val_loss: 1.1917 - val_2l

Epoch 23/40
946/946 [==============================] - 72s 76ms/step - loss: 0.4946 - 2logo_center_loss: 0.2479 - 2logo_left_loss: 0.2467 - 2logo_center_binary_accuracy: 0.8975 - 2logo_left_binary_accuracy: 0.8975 - val_loss: 1.2287 - val_2logo_center_loss: 0.6153 - val_2logo_left_loss: 0.6134 - val_2logo_center_binary_accuracy: 0.8005 - val_2logo_left_binary_accuracy: 0.8021
Epoch 24/40
946/946 [==============================] - 76s 80ms/step - loss: 0.4944 - 2logo_center_loss: 0.2478 - 2logo_left_loss: 0.2466 - 2logo_center_binary_accuracy: 0.8975 - 2logo_left_binary_accuracy: 0.8976 - val_loss: 1.2303 - val_2logo_center_loss: 0.6161 - val_2logo_left_loss: 0.6142 - val_2logo_center_binary_accuracy: 0.8003 - val_2logo_left_binary_accuracy: 0.8020
Epoch 25/40
946/946 [==============================] - 75s 79ms/step - loss: 0.4941 - 2logo_center_loss: 0.2477 - 2logo_left_loss: 0.2464 - 2logo_center_binary_accuracy: 0.8977 - 2logo_left_binary_accuracy: 0.8977 - val_loss: 1.2319 - val_2lo

In [18]:
# efficientnet_v2_step2.save_weights('/tf/datasets/step2_multi/'+target+'/'+target)
efficientnet_v2_step2.save('/tf/datasets/step2_multi/'+target+'/'+str(version))

INFO:tensorflow:Assets written to: /tf/datasets/step2_multi/header/3/assets


INFO:tensorflow:Assets written to: /tf/datasets/step2_multi/header/3/assets


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
results = efficientnet_v2_step2.evaluate(testable_ds)

# 결과 출력
print("Loss:", results[0])
print("Accuracy:", results[1])

In [ ]:
import matplotlib.pyplot as plt

# 훈련과정에서 계산한 손실값과 정확도값을 가져옴
loss = history.history['loss']
accuracy = history.history['val_LOGO_L_binary_accuracy']

# 손실 그래프
plt.plot(loss, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

# 정확도 그래프
plt.plot(accuracy, label='Training Accuracy')
pl
t.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np

# 테스트 데이터셋을 사용하여 모델 예측
predictions = efficientnet_v2_step2.predict(testable_ds)

# 예측 결과 출력
for i, label in enumerate(labels):
    print(f"{label} - Prediction: {predictions[i]}")